This notebook is used to save the tidal parameters calculated from the model output to a csv file in order to avoid having to load all the files run.

In [1]:
import os
import glob
import datetime
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import re
from dateutil import tz

import netCDF4 as nc
import scipy.io as sio
from scipy.optimize import curve_fit
from salishsea_tools import (viz_tools,tidetools, nc_tools, ellipse)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline


#Hourly Data
##@ VENUS nodes
The functions below will facilitate loading the hourly data and writting the tidal parameters to a csv file.

Function to save tidal parameters in a csv file.

In [2]:
def writetocsv(runname, depth, major, minor,theta, phase, majork1, minork1, thetak1, phasek1, station):
    outfile = runname+'.csv'
    
    if station == 'Central':
        k=0
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[k][i], major[i,k], minor[i,k], theta[i,k], phase[i,k], majork1[i,k], minork1[i,k], thetak1[i,k], phasek1[i,k]])

    elif station == 'East':
        k=1
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[k][i], major[i,k], minor[i,k], theta[i,k], phase[i,k], majork1[i,k], minork1[i,k], thetak1[i,k], phasek1[i,k]])

    elif major.shape[0] == 40:
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[i], major[i], minor[i], theta[i], phase[i], majork1[i], minork1[i], thetak1[i], phasek1[i]])

    else:
        k=0
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            mat = (major[0], minor[0], theta[0], phase[0], majork1[0], minork1[0], thetak1[0], phasek1[0])
            writer.writerow(mat)


In [19]:
def writetocsv_new(runname, params, dep='None'):
    outfile = runname+'.csv'
    
    if dep == 'None':
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            mat = ([params['M2']['Semi-Major Axis'][0,0], params['M2']['Semi-Minor Axis'][0,0], params['M2']['Inclination'][0,0], params['M2']['Phase'][0,0], 
                       params['K1']['Semi-Major Axis'][0,0], params['K1']['Semi-Minor Axis'][0,0], params['K1']['Inclination'][0,0], params['K1']['Phase'][0,0]])
            writer.writerow(mat)

    else:
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([dep[i], params['M2']['Semi-Major Axis'][i,0][0], params['M2']['Semi-Minor Axis'][i,0][0], params['M2']['Inclination'][i,0][0], params['M2']['Phase'][i,0][0],params['K1']['Semi-Major Axis'][i,0][0], params['K1']['Semi-Minor Axis'][i,0][0], params['K1']['Inclination'][i,0][0], params['K1']['Phase'][i,0][0]])


###All the hourly data

Do not use data before November 26th 2014 for tidal ellipses, the model tides change this day.

In [23]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,7,13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depav = [35,300,20,160]

Central and East depth averaged without surface and bottom effects

In [24]:
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2, depthrange=[35,300], depav=True)
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Nodes/{}_{}_Central_depav'.format(t_o, t_f)


/data/dlatorne/MEOPAR/SalishSea/nowcast/26nov14/SalishSea_1h_20141126_20141126_grid_U.nc


IOError: [Errno 2] No such file or directory: '/ocean/nsoontie/TidalEllipseData/Nodes/26nov14_13jul15_Central_depav.csv'

In [27]:
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Nodes/{}_{}_Central_depav'.format(t_o, t_f)
writetocsv_new(runname1, params)

In [16]:
params, depth = ellipse.get_params_nowcast(to, tf, [283], [416],  path, 2,depthrange=[20,160], depav=True)
runname2 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Nodes/{}_{}_East_depav'.format(t_o, t_f)
writetocsv_new(runname2, params)

Central and east whole water column

In [36]:
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2)
runname1 = 'TidalEllipseData/Nodes/{}_{}_Central'.format(t_o, t_f)
writetocsv_new(runname1, params, dep=depth)

In [38]:
params, depth = ellipse.get_params_nowcast(to, tf, [283], [416], path, 2)
runname2 = 'TidalEllipseData/Nodes/{}_{}_East'.format(t_o, t_f)
writetocsv_new(runname2, params, dep=depth)

###Hourly data without the freshet

In [15]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,4,26)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depav = [35,300,20,160]

In [18]:
freq = 'h'
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2)
runname1 = 'TidalEllipseData/Nodes/{}_{}_{}_Central'.format(t_o, t_f, 'h')
writetocsv_new(runname1, params, dep=depth)

In [16]:
params, depth = ellipse.get_params_nowcast(to, tf, [283], [416],  path, 2)
runname2 = 'TidalEllipseData/Nodes/{}_{}_{}_East'.format(t_o, t_f, 'h')
writetocsv_new(runname2, params, dep=depth)

In [20]:
print 'done'
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2, depthrange=[35, 300], depav=True)
print 'done'

done
done


In [21]:
runname2 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Nodes/{}_{}_{}_Central-depav'.format(t_o, t_f, 'h')
writetocsv_new(runname2, params)

params, depth = ellipse.get_params_nowcast(to, tf, [283], [416],  path, 2, depthrange=[20, 160], depav=True)
print 'done'
runname2 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Nodes/{}_{}_{}_East-depav'.format(t_o, t_f, 'h')
writetocsv_new(runname2, params)

done


###Seasonal sensitivity
Winter

In [23]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,2,26)
freq = 'h'
t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2)
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_Central_winter'.format(t_o, t_f)
writetocsv_new(runname1, params, dep=depth)

params, depth = ellipse.get_params_nowcast(to, tf, [283], [416], path, 2)
runname2 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_East_winter'.format(t_o, t_f)
writetocsv_new(runname2, params, dep=depth)

Spring

In [22]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,2,27)
tf=datetime.datetime(2015,5,27)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2)
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_Central_spring'.format(t_o, t_f)
writetocsv_new(runname1, params, dep=depth)

params, depth = ellipse.get_params_nowcast(to, tf, [283], [416], path, 2)
runname2 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_East_spring'.format(t_o, t_f)
writetocsv_new(runname2, params, dep=depth)

/data/dlatorne/MEOPAR/SalishSea/nowcast/27feb15/SalishSea_1h_20150227_20150227_grid_U.nc
/data/dlatorne/MEOPAR/SalishSea/nowcast/27feb15/SalishSea_1h_20150227_20150227_grid_U.nc


###For all quarter-hourly values

In [18]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,4,26)
freq= 'h'

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

params, depth = ellipse.get_params_nowcast(to, tf, [266], [424], path, 2)
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_{}_Central'.format(t_o, t_f, freq)
writetocsv_new(runname1, params, dep=depth)

params, depth = ellipse.get_params_nowcast(to, tf, [283], [416], path, 2)
runname2 = 'TidalEllipseData/Sensitivity/{}_{}_{}_East'.format(t_o, t_f, freq)
writetocsv_new(runname2, params, dep=depth)

/data/dlatorne/MEOPAR/SalishSea/nowcast/26nov14/SalishSea_1h_20141126_20141126_grid_U.nc


NameError: name 'writetocsv_new' is not defined

In [21]:
runname1 = '/ocean/nsoontie/MEOPAR/TidalEllipseData/Sensitivity/{}_{}_{}_Central'.format(t_o, t_f, freq)
writetocsv_new(runname1, params, dep=depth)

Then to open and read the columns you could use the lines below:

df = pd.read_csv('20150601_20150608_h_Central.csv')
depth = df.Depth

#Quarter-Hourly Data
##@ VENUS nodes

### The Month of May 2015

The gridded data quarter hourly data only started to be recorded on May 9th.

In [13]:
reload(ellipse)

<module 'salishsea_tools.ellipse' from '/data/nsoontie/MEOPAR/tools/SalishSeaTools/salishsea_tools/ellipse.py'>

In [16]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,5,9) #changed to be able to load gridded files.
tf=datetime.datetime(2015,7,13)
freq='h'

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

params_east = ellipse.get_params_nowcast_15(to,tf,'east',path,8)

#depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k=loadparam(to, tf, path)
#runname1 = 'TidalEllipseData/Sensitivity/{}_{}_{}_Central'.format(t_o, t_f, freq)
#runname2 = 'TidalEllipseData/Sensitivity{}_{}_{}_East'.format(t_o, t_f, freq)

#writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'Central')
#writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'East')

/data/dlatorne/MEOPAR/SalishSea/nowcast/09may15/VENUS_east_gridded.nc


#Hourly Data at Other Locations
##Foreman Model Locations
In Foreman et al. (2004) the tidal currents amplitude and phase of their model was compared to observations at 7 locations. For the locations that are contained in the model I will make comparisons with these values. I this notebook I will load the data. See [notebook](http://nbviewer.ipython.org/urls/bitbucket.org/salishsea/analysis/raw/tip/Muriel/TidalEllipsesComparisons.ipynb) for comparison. The locations and numbers are from Foreman et al. (2004)

###35 - Haro Strait

In [4]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'
lon35 = -123.225789
lat35 = 48.537579

grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
bathy, X, Y = tidetools.get_bathy_data(grid_B)
j35, i35 = tidetools.find_closest_model_point(lon35, lat35, X, Y, bathy)
print i35, j35

233 312


In [ ]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depav = [35,300,20,160]

params, dep = ellipse.get_params_nowcast(to, tf, [i35], [j35], path, 2)
runname1 = '/ocean/nsoontie/TidalEllipseData/ForemanLocation/{}_{}_Haro'.format(t_o, t_f)

In [22]:
writetocsv_new(runname1, params, dep=dep)

In [8]:
params, dep = ellipse.get_params_nowcast(to, tf, [i35], [j35], path, 2, depthrange=[0, 400], depav=True)
runname2 = '/ocean/nsoontie/TidalEllipseData/ForemanLocation/{}_{}_Haro_depav(0-400)'.format(t_o, t_f)
writetocsv_new(runname2, params, dep='None')

###33 - Juan de Fuca West

In [17]:
lat33 = 48.483
lon33 = -124.713

j33, i33 = tidetools.find_closest_model_point(lon33, lat33, X, Y, bathy)
print i33, j33

8 408


In [18]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()


depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i33, j33)
runname1 = 'TidalEllipseData/ForemanLocation/{}_{}_JuanW'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [19]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i33, j33, depav=[0, 400])
runname2 = 'TidalEllipseData/ForemanLocation/{}_{}_JuanW_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###34 - Juan de Fuca East

In [20]:
lat34 = 48.232
lon34 = -123.530

j34, i34 = tidetools.find_closest_model_point(lon34, lat34, X, Y, bathy)
print i34, j34

151 275


In [21]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i34, j34)
runname1 = 'TidalEllipseData/ForemanLocation/{}_{}_JuanE'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [22]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i34, j34, depav=[0, 400])
runname2 = 'TidalEllipseData/ForemanLocation/{}_{}_JuanE_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###36 - Seymour Narrows
Seymour Narrows is too narrow. Therefore we encounter problems when unstaggering. The neighbouring cells are masked and this causes problems in the calculation routines. I will choose a slightly different location while trying to maintain similar bathymetry and velocities.

In [23]:
lat36 = 50.135820
lon36 = -125.353403

j36, i36 = tidetools.find_closest_model_point(lon36, lat36, X, Y, bathy)
print i36, j36, bathy[j36, i36]

120 774 54.25


In [24]:
inew36=i36-1
jnew36=j36-2

In [25]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew36, jnew36)
runname1 = 'TidalEllipseData/ForemanLocation/{}_{}_SN'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [26]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k=research_VENUS.loadparam_all(to, tf, path, inew36, jnew36, depav=[0, 400])
runname2 = 'TidalEllipseData/ForemanLocation/{}_{}_SN_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###38 - Johnstone Strait

In [27]:
lat38 = 50.454991
lon38 = -126.038740

j38, i38 = tidetools.find_closest_model_point(lon38,lat38,X, Y, bathy)
print i38, j38

58 885


In [28]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i38, j38)
runname1 = 'TidalEllipseData/ForemanLocation/{}_{}_JS'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i38, j38, depav=[0, 400])
runname2 = 'TidalEllipseData/ForemanLocation/{}_{}_JS_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')


###37 - Arran Rapids

In [29]:
lat37 = 50.416667
lon37 = -125.133333
j37, i37 = tidetools.find_closest_model_point(lon37,lat37,X, Y, bathy)
print i37, j37

189 814


In [30]:
inew37 = i37+2
jnew37 = j37-1

In [6]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)


t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

In [32]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew37, jnew37)
runname1 = 'TidalEllipseData/ForemanLocation/{}_{}_AR'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew37, jnew37, depav=[0, 500])
runname2 = 'TidalEllipseData/ForemanLocation/{}_{}_AR_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

##Thalweg Locations

In [4]:
thalweg = np.loadtxt('/data/dlatorne/MEOPAR/tools/bathymetry/thalweg_working.txt', dtype=int, unpack=True)

##Systematic choice of Thalweg Locations

In [5]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

In [9]:
for k in [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1303, 1400, 1500]:
    i = thalweg[1,k]
    j = thalweg[0,k]
    param_thal, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2)
    runname1 = 'TidalEllipseData/Thalweg/{}_{}_thalweg_{}'.format(t_o, t_f, k)
    writetocsv_new(runname1, param_thal, dep=depth)
    print 'done depth column'
    param_thal_depav, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2, depthrange=[20, 400], depav=True)
    runname2 = 'TidalEllipseData/Thalweg/{}_{}_thalweg_{}_depav(20-400)'.format(t_o, t_f, k)
    writetocsv_new(runname2, param_thal_depav, dep='None')
    print k

done depth column
0
done depth column
100
done depth column
200
done depth column
300
done depth column
400
done depth column
500
done depth column
600
done depth column
700
done depth column
800
done depth column
900
done depth column
1000
done depth column
1100
done depth column
1200
done depth column
1303
done depth column
1400
done depth column
1500


##Haro

In [10]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)


t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

for k in np.arange(480,520,4):
    i = thalweg[1,k]
    j = thalweg[0,k]
    param_har, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2)
    runname1 = 'TidalEllipseData/HaroStrait/{}_{}_HaroS_{}'.format(t_o, t_f, k)
    writetocsv_new(runname1, param_har, dep=depth)
    print 'done depth column'
    param_har_depav, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2, depthrange=[20, 400], depav=True)
    runname2 = 'TidalEllipseData/HaroStrait/{}_{}_HaroS_{}_depav(20-400)'.format(t_o, t_f, k)
    writetocsv_new(runname2, param_har_depav, dep='None')


done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column


#Juan de Fuca

In [22]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

for k in np.arange(0,350,25):
    i = thalweg[1,k]
    j = thalweg[0,k]
    param_jdf, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2)
    runname1 = 'TidalEllipseData/JuandeFuca/{}_{}_JdF_{}'.format(t_o, t_f, k)
    writetocsv_new(runname1, param_jdf, dep=depth)
    print 'done depth column'
    
for k in np.arange(175,350,25):
    i = thalweg[1,k]
    j = thalweg[0,k]
    param_jdf_depav, depth= ellipse.get_params_nowcast(to, tf, [i], [j], path, 2, depthrange=[20, 400], depav=True)
    runname2 = 'TidalEllipseData/JuandeFuca/{}_{}_JdF_{}_depav(20-400)'.format(t_o, t_f, k)
    writetocsv_new(runname2, param_jdf_depav, dep='None')
    print k

done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
done depth column
175
200
225
250
275
300
325


#CODAR

In [3]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 7, 13)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

In [4]:
jmin = 379; jmax = 461; imin = 236; imax=321
j = np.arange(jmin,jmax)
i = np.arange(imin,imax)
params, depth = ellipse.get_params_nowcast(to, tf, i, j, path, 2, depthrange=1)

###Save values into netCDF file

netCDF dataset for M2 CODAR region.

In [5]:
A = nc.Dataset('/ocean/nsoontie/MEOPAR/TidalEllipseData/CODAR/CODARM2param.nc', 'w')
A.createDimension('y', len(j))
A.createDimension('x', len(i))

<type 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 85

In [6]:
grid = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
bathy = grid.variables['Bathymetry']
lons_grid = grid.variables['nav_lon']
lats_grid = grid.variables['nav_lat']


In [7]:
lons = A.createVariable('nav_lon', float, ('y','x'), zlib=True)
lons[:] = lons_grid[j,i]
lats = A.createVariable('nav_lat', float, ('y','x'), zlib=True)
lats[:] = lats_grid[j,i]

In [8]:
sema = A.createVariable('Semi-Major', float, ('y','x'), zlib=True)
sema[:] = params['M2']['Semi-Major Axis']
semi = A.createVariable('Semi-Minor', float, ('y','x'), zlib=True)
semi[:] = params['M2']['Semi-Minor Axis']
inc = A.createVariable('Inclination', float, ('y','x'), zlib=True)
inc[:] = params['M2']['Inclination']
pha = A.createVariable('Phase', float, ('y','x'), zlib=True)
pha[:] = params['M2']['Phase']
sema.units = 'm/s'
semi.units = 'm/s'
inc.units = 'deg CCW E'
pha.units = ' deg'
A.title = 'M2 ellipse parameter at 0.5m in the CODAR region {} to {}'.format(t_o, t_f)
A.close()

netCDF dataset for K1 CODAR region

In [9]:
B = nc.Dataset('/ocean/nsoontie/MEOPAR/TidalEllipseData/CODAR/CODARK1param.nc', 'w')
B.createDimension('y', len(j))
B.createDimension('x', len(i))

<type 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 85

In [10]:
lons = B.createVariable('nav_lon', float, ('y','x'), zlib=True)
lons[:] = lons_grid[j,i]
lats = B.createVariable('nav_lat', float, ('y','x'), zlib=True)
lats[:] = lats_grid[j,i]

In [11]:
sema = B.createVariable('Semi-Major', float, ('y','x'), zlib=True)
sema[:] = params['K1']['Semi-Major Axis']
semi = B.createVariable('Semi-Minor', float, ('y','x'), zlib=True)
semi[:] = params['K1']['Semi-Minor Axis']
inc = B.createVariable('Inclination', float, ('y','x'), zlib=True)
inc[:] = params['K1']['Inclination']
pha = B.createVariable('Phase', float, ('y','x'), zlib=True)
pha[:] = params['K1']['Phase']
sema.units = 'm/s'
semi.units = 'm/s'
inc.units = 'deg CCW E'
pha.units = ' deg'
B.title = 'K1 ellipse parameter at 0.5m in the CODAR region {} to {}'.format(t_o, t_f)
B.close()